***GENERATED CODE FOR 3110sqlsink1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 3110sqlsink1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	bostonhousingtest = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Boston_Housing_Test.csv', 'filename': 'Boston_Housing_Test.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run 3110sqlsink1Hooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bostonhousingtest,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/Boston_Housing_Test.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/Boston_Housing_Test.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/6540d74b730bd4ed2218f2131698748348/0part.csv", "requestRatio": 0.0, "totalRows": 19, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 14, "numberOfRows": 19, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 0.19539, "min": 0.01951, "mean": 0.08316473684210526, "missing": 0.0, "stddev": 0.05, "outliers": [0.19539], "validation": []}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 80.0, "min": 0.0, "mean": 15.263157894736842, "missing": 0.0, "stddev": 24.75, "outliers": [80.0, 80.0], "validation": []}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 12.83, "min": 1.38, "mean": 8.480526315789476, "missing": 0.0, "stddev": 4.05, "outliers": [], "validation": []}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "numeric", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0, "outliers": [], "validation": []}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.437, "min": 0.398, "mean": 0.4205842105263158, "missing": 0.0, "stddev": 0.01, "outliers": [], "validation": []}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 7.104, "min": 5.594, "mean": 6.2083157894736845, "missing": 0.0, "stddev": 0.36, "outliers": [7.104], "validation": []}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 74.5, "min": 6.0, "mean": 33.44210526315789, "missing": 0.0, "stddev": 21.05, "outliers": [], "validation": []}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 9.2229, "min": 4.0522, "mean": 5.563405263157895, "missing": 0.0, "stddev": 1.23, "outliers": [9.2229], "validation": []}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "numeric", "max": 5.0, "min": 3.0, "mean": 4.2631578947368425, "missing": 0.0, "stddev": 0.56, "outliers": [], "validation": []}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "numeric", "max": 398.0, "min": 216.0, "mean": 335.57894736842104, "missing": 0.0, "stddev": 53.03, "outliers": [], "validation": []}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 19.2, "min": 16.1, "mean": 18.605263157894736, "missing": 0.0, "stddev": 0.91, "outliers": [16.1, 16.1], "validation": []}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 373.66, "mean": 390.34, "missing": 0.0, "stddev": 7.94, "outliers": [], "validation": []}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 13.09, "min": 4.67, "mean": 8.48578947368421, "missing": 0.0, "stddev": 2.39, "outliers": [], "validation": []}, {"column": "ZN_Indus", "alias": "ZN_Indus", "generated": 1, "type": "real", "max": 83.37, "min": 10.81, "mean": 23.743684210526318, "missing": 0.0, "stddev": 22.0, "outliers": [83.37, 83.37], "validation": []}]}, "predictionPowerScore": [{"AGE": 1.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0, "ZN_Indus": 0.0}, {"AGE": 0.0, "B": 1.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.1201990432, "INDUS": 0.3999604903, "LSTAT": 0.0, "NOX": 0.3163708782, "PTRATIO": 0.3901407398, "RAD": 0.0193826799, "RM": 0.0, "TAX": 0.4033960505, "ZN": 0.3982378796, "ZN_Indus": 0.4018270545}, {"AGE": 0.0, "B": 0.0, "CHAS": 1.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "LSTAT": 0.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0, "ZN_Indus": 0.0}, {"AGE": 0.0, "B": 0.0259329438, "CHAS": 0.00742728, "CRIM": 1.0, "DIS": 0.030499265, "INDUS": 0.1514686581, "LSTAT": 0.0, "NOX": 0.0, "PTRATIO": 0.0841601293, "RAD": 0.0, "RM": 0.0, "TAX": 0.1500935606, "ZN": 0.0, "ZN_Indus": 0.0554517035}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 1.0, "INDUS": 0.7297822417, "LSTAT": 0.0, "NOX": 0.6504968246, "PTRATIO": 0.5902773489, "RAD": 0.2268031939, "RM": 0.0, "TAX": 0.657286559, "ZN": 0.5282218023, "ZN_Indus": 0.7229865199}, {"AGE": 0.1108138686, "B": 0.1174222628, "CHAS": 0.0, "CRIM": 0.4815565693, "DIS": 0.9443868613, "INDUS": 1.0, "LSTAT": 0.0, "NOX": 0.8692189781, "PTRATIO": 0.8412043796, "RAD": 0.4418648187, "RM": 0.1229835766, "TAX": 0.9517372263, "ZN": 0.785211505, "ZN_Indus": 0.9349562044}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "LSTAT": 1.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0, "ZN_Indus": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.8222444444, "INDUS": 0.9235777778, "LSTAT": 0.0, "NOX": 1.0, "PTRATIO": 0.9117555556, "RAD": 0.5772809203, "RM": 0.0, "TAX": 0.9582, "ZN": 0.4283714286, "ZN_Indus": 0.9700222222}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.6686046512, "INDUS": 0.7238372093, "LSTAT": 0.0, "NOX": 0.9337209302, "PTRATIO": 1.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.9558139535, "ZN": 0.6716292913, "ZN_Indus": 0.9668604651}, {"AGE": 0.1517857143, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1857142857, "DIS": 0.7285714286, "INDUS": 0.8642857143, "LSTAT": 0.0, "NOX": 0.8642857143, "PTRATIO": 0.7285714286, "RAD": 1.0, "RM": 0.0, "TAX": 0.8642857143, "ZN": 0.1388605442, "ZN_Indus": 0.8642857143}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0510623677, "LSTAT": 0.0, "NOX": 0.0839876562, "PTRATIO": 0.0889183398, "RAD": 0.0, "RM": 1.0, "TAX": 0.1504870074, "ZN": 0.0301818803, "ZN_Indus": 0.0018528478}, {"AGE": 0.0332249071, "B": 0.0, "CHAS": 0.0, "CRIM": 0.2183395291, "DIS": 0.7480793061, "INDUS": 0.85755886, "LSTAT": 0.0, "NOX": 0.8952292441, "PTRATIO": 0.7857496902, "RAD": 0.5281876116, "RM": 0.0140954151, "TAX": 1.0, "ZN": 0.2629469818, "ZN_Indus": 0.8481412639}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.7952586207, "INDUS": 0.7952586207, "LSTAT": 0.0, "NOX": 0.9426724138, "PTRATIO": 0.9426724138, "RAD": 0.0381970879, "RM": 0.0, "TAX": 0.9754310345, "ZN": 1.0, "ZN_Indus": 0.9836206897}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.7173205977, "INDUS": 0.7259059592, "LSTAT": 0.0, "NOX": 0.9657010558, "PTRATIO": 0.9742864173, "RAD": 0.0, "RM": 0.0, "TAX": 0.9533330351, "ZN": 0.9527916432, "ZN_Indus": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "ZN", "type": "real", "min": "0.0", "max": "80.0", "mean": "15.3"}, {"columnName": "INDUS", "type": "real", "min": "1.4", "max": "12.8", "mean": "8.5"}], "functionsData": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "80.0", "mean": "15.3"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "1.4", "max": "12.8", "mean": "8.5"}}, {"Columndata": ""}], "newColumnName": "ZN_Indus"}]}], "functionChanges": [{"columnName": "ZN", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "80.0", "mean": "15.3"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "1.4", "max": "12.8", "mean": "8.5"}}, {"Columndata": ""}], "newColumnName": "ZN_Indus"}]}, {"columnName": "INDUS", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "80.0", "mean": "15.3"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "1.4", "max": "12.8", "mean": "8.5"}}, {"Columndata": ""}], "newColumnName": "ZN_Indus"}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "numeric"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "numeric"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "numeric"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "ZN_Indus", "alias": "ZN_Indus", "generated": 1, "position": 14, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run 3110sqlsink1Hooks.ipynb
try:
	#sinkPreExecutionHook()

	sqlserver = RDBMSConnector.put(spark, "{'host': '3.17.87.4', 'port': '1433', 'password': 'Passw0rd@123', 'database': 'testdb', 'user': 'SA', 'optionalDB': '', 'table_option': 'create_new_table', 'table': 'NumtraSQLS2', 'new_table_name': 'NumtraSQLS2', 'writemode': '', 'is_header': 'Use Header Line', 'use_generatedColumns': False, 'generatedColumns': [], 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sinkPostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)
